In [ ]:
# !pip install ultralytics==8.1.29
# !pip install easyocr
# !pip install rich

# TEXT MESSAGE TO JSON

In [3]:
# import libraries and load detection model

from PIL import Image
from ultralytics import YOLO
import easyocr
import cv2
from warnings import filterwarnings

import numpy as np
import re
import json
import glob
import os
import shutil
import paddleocr
from paddleocr import PaddleOCR
import pytesseract
from pylab import array, plot, show, axis, arange, figure, uint8
from rich import print

filterwarnings('ignore')

# Create a reader to do OCR.
reader = easyocr.Reader(['en'])

# Load a pretrained YOLOv8n model
# model = YOLO('yolo_models/large_best_200eph.pt', verbose=False) # 4 star
model = YOLO('yolo_models/yolov8l_imgsz800_map85_best.pt', verbose=False)
# pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'
print('Model Loaded...')

Model Loaded...

In [7]:
import re
import json
import cv2
import numpy as np
import os
import shutil
# import pytesseract

ocr_model = PaddleOCR(use_angle_cls=True, lang='en', show_log=False,
                      rec_model_dir='rec_r45_abinet', det_model_dir='det_r50_vd_east_v2.0')

if os.path.exists('cropped'):
    shutil.rmtree('cropped/')


def perform_pad_ocr(image, conf=0.20, iou=.91, imgsz=800):    
    # make detection
    test_file = image
    results = model.predict(test_file, conf=conf, iou=iou, imgsz=imgsz) # results = model.predict(test_file, conf=0.285, iou=0.885, imgsz=800)
    ocr_result_path = "OCR_RESULTS"
    image_fname = test_file.split('/')[-1]
    ocr_lst = []
    temp_timestamp = ""

    chat = []
    chat1 = []

    timestamp_formats = ['%I.%M %p', '%I %p', '%I.%M%p', '%I%M %p', '%I %p', '%I %M %p',
                         '%I:%M %p', '%I:%M:%S %p', '%H:%M', '%H.%M', '%H %M', '%H:%M:%S', '%H.%M.%S',
                         '%H %M %S', '%H:%M:%S.%f', '%H.%M.%S.%f', '%H %M %S %f']

    for j in results:
        os.makedirs(ocr_result_path, exist_ok=True)
        boxes = j.boxes.cpu().numpy()
        j.save(filename=f'{ocr_result_path}/{image_fname}_result.jpg')  # save to disk
        
        j.save_crop('cropped')

        message_boxes = [box for box in boxes if j.names[int(box.cls[0])] in ['sender_message', 'receiver_message']]
        replied_boxes = [box for box in boxes if j.names[int(box.cls[0])] in ['replied_to_message']]
        message_boxes.sort(key=lambda box: box.xyxy[0][1])
        replied_boxes.sort(key=lambda box: box.xyxy[0][1])
        emoji_boxes = [box for box in boxes if j.names[int(box.cls[0])] == 'emoji']
        timestamp_boxes = [box for box in boxes if j.names[int(box.cls[0])] == 'timestamp']
        
        emoji_text = " "
        emo_lst = []
        emoji_txt_lst = []
        
        


        for msg_box, t_box in zip(message_boxes, timestamp_boxes):
            message_type = j.names[int(msg_box.cls[0])]
            # print(message_type)
            msg_r = msg_box.xyxy[0].astype(int)
            roi = (cv2.imread(test_file, 0))[msg_r[1]:msg_r[3], msg_r[0]:msg_r[2]]
            ocr_res = ocr_model.ocr(roi, cls=True)[0]
            ocr_text = " "
            ocr_text3 = " "
            new_entries = []
            temp_text = []
            entries_with_positions = []
            if ocr_res is not None:
                for i in ocr_res:
                    txt = i[-1][0]
                    txt = txt.replace('\n', ' ')
                    txt = txt.strip()
                    ocr_text += f"{txt} "
                    # new_entries.append(txt)
                    # entries_with_positions.append((msg_r[0], txt))  # Add position
                    # entries_with_positions.append(((msg_r[0], msg_r[1]), txt))  # Add position as tuple
                        
                
                t_i = cv2.imread(test_file)
                emoji_positions = []
                fin_oc = []
    
                ocr_text1 = " "
                
                
                    
                for i,emoji_box in enumerate(emoji_boxes):
                    emoji_r = emoji_box.xyxy[0].astype(int)
                    kernel = np.ones((8,8),np.float32)/15
                    
                    t_i[emoji_r[1]:emoji_r[3], emoji_r[0]:emoji_r[2]] = cv2.filter2D(t_i[emoji_r[1]:emoji_r[3], emoji_r[0]:emoji_r[2]], -1, kernel)
                    
                    center_x = int((emoji_r[0] + emoji_r[2]) / 2)
                    center_y = int((emoji_r[1] + emoji_r[3]) / 2)
                    
                    # Adjust the text position so that the center of the text aligns with the center of the bounding box
                    text_size = cv2.getTextSize("%", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
                    text_x = center_x - text_size[0] // 2
                    text_y = center_y + text_size[1] // 2
                    
                    # cv2.putText(t_i, "MM ", (emoji_r[0]+5, emoji_r[1]+27), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0) , 2) # (36,255,12)
                    cv2.putText(t_i, "% ", (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0) , 2)
                    cv2.imwrite('t_i.jpg', t_i)
    
    
                    roi1 = (t_i[msg_r[1]:msg_r[3], msg_r[0]:msg_r[2]])
                    ocr_res1 = ocr_model.ocr(roi1, cls=True)[0]
                    new_oc = []
                    if ocr_res1 is not None:
                        for i in ocr_res1:
                            txt1 = i[-1][0]
                            txt1 = txt1.replace('\n', ' ')
                            txt1 = txt1.strip()
                            ocr_text1 += f"{txt1} "
                            new_oc.append(txt1)
                    fin_oc.append(new_oc)
                    
                ocr_text2 = " ".join(fin_oc[-1])
                ocr_text3 = ocr_text2
                ocr_text3 = re.sub(r'(EA|EF|EL|EM|EI|EN|EV|El|FM|FN|FI|FA|FE|FV|M E|E M|EE|MM|NM|MN|MI|NN|IM|M1|1M|MW|AN|SN|Q|%)+', '$$ ', ocr_text3)
                # print(ocr_text2)

            rep_tt = ""           
            for replied_box in replied_boxes:
                rep_r = replied_box.xyxy[0].astype(int)
                if (msg_r[0] <= rep_r[0] and rep_r[2] <= msg_r[2]) and (msg_r[1] <= rep_r[1] and msg_r[3] <= rep_r[3]):
                    rep_roi = cv2.imread(test_file, 0)[rep_r[1]:rep_r[3], rep_r[0]:rep_r[2]]
                    rep_ocr_res = ocr_model.ocr(rep_roi, cls=True)[0]
                    # print(rep_ocr_res)
                    if rep_ocr_res is not None:
                        for i in rep_ocr_res:
                            txt = i[-1][0]
                            rep_tt += f"{txt} "

            # print(rep_tt)

                        
            try:
                for time_box in timestamp_boxes:
                    time_r = time_box.xyxy[0].astype(int)
                    if (msg_r[0] <= time_r[0] and time_r[2] <= msg_r[2]) and (msg_r[1] <= time_r[1] and time_r[3] <= msg_r[3]):
                        ts_r = time_r
                        ts_roi = cv2.imread(test_file)[ts_r[1]:ts_r[3], ts_r[0]:ts_r[2]]
                        ts_ocr_res = ocr_model.ocr(ts_roi, cls=True)[0]
                        ts_text = ""
                        if ts_ocr_res is not None:
                            for i in ts_ocr_res:
                                txt = i[-1][0]
                                ts_text += txt
                                timestamp = txt
                if timestamp is None:
                    # match = re.search(r'\d{1,2}:\d{2} (?:AM|PM)|\d{1,5} (?:AM|PM)', ocr_text)
                    # match = re.search(r'\d{1,2}:\d{2}\s?(?:AM|PM)|\d{1,5}\s?(?:AM|PM)', ocr_text)
                    match = re.search(r'\b\d{1,2}:\d{2}\s?(?:AM|PM)\b', ocr_text)
                    timestamp = match.group() if match else None                
                
                # Remove timestamp from ocr_text
                ocr_text = ocr_text.replace(timestamp, '') if timestamp else ocr_text
                ocr_text = re.sub(r'\b\d{1,2}:\d{2}\s?(?:AM|PM)\b', '', ocr_text)
                
                # Remove replied text from ocr_text
                ocr_text = ocr_text.replace(rep_tt, '') if rep_tt else ocr_text
    
    
    
                
                        
                dictt = {"type": message_type, "text": ocr_text.strip(), 'timestamp': timestamp}
                chat.append(dictt)
                # for f in temp_text:
        
                # Remove timestamp from ocr_text1
                ocr_text3 = ocr_text3.replace(timestamp, ' ') if timestamp else ocr_text3
                ocr_text3 = re.sub(r'\b\d{1,2}:\d{2}\s?(?:AM|PM)\b', '', ocr_text3)
                ocr_text3 = ocr_text1.replace(rep_tt, '') if rep_tt else ocr_text3
                # print(ocr_text3)
                dictt1 = {"type": message_type, "text": ocr_text3.strip(), 'timestamp': timestamp}
                chat1.append(dictt1)
            except:
                pass
            

            
    json.dump(chat, open(f"{ocr_result_path}/{image_fname}_result.json", 'w'))
    json.dump(chat1, open(f"{ocr_result_path}/{image_fname}_result1.json", 'w'))
    return chat1

In [10]:
all_tests = sorted(glob.glob('test_images/*'))
print(len(all_tests))
for test_img in all_tests:
    print('\n\n\tworking on: :', test_img)
    print(perform_pad_ocr(test_img, conf=0.20, iou=0.965, imgsz=832))

4

working on: : test_images/ss_test1.png


image 1/1 /Users/mac/fivv/next_sapien/yolov8/Next_sapien_V3/test_images/ss_test1.png: 832x480 23 emojis, 17 message_status_indicators, 4 receiver_messages, 4 sender_messages, 26 timestamps, 647.4ms
Speed: 3.5ms preprocess, 647.4ms inference, 0.4ms postprocess per image at shape (1, 3, 832, 480)


[
    {'type': 'receiver_message', 'text': "Hey there!. $$  How's your day going?e", 'timestamp': '7:38AM'},
    {
        'type': 'sender_message',
        'text': "Hey! $$  It's going pretty well,. just catching up on some work.",
        'timestamp': '7:38AM'
    },
    {'type': 'receiver_message', 'text': 'Nice! $$  Any plans for the weekend?', 'timestamp': '7:38AM'},
    {
        'type': 'sender_message',
        'text': "Yeah, I'm thinking of going for a hike $$  with some friends. How. about you?",
        'timestamp': '7:38AM'
    },
    {
        'type': 'receiver_message',
        'text': "Oh, that sounds fun! $$  I'm probably just going to relax at home $$  and binge-watch some Netflix
shows. $$",
        'timestamp': '7:38AM'
    },
    {
        'type': 'sender_message',
        'text': 'Sounds like a plan!. What are you watching these days?',
        'timestamp': '7:39AM'
    },
    {
        'type': 'receiver_message',
        'text': "I started watching that new mystery series $$  everyone's talking about. It's pretty intense!",
        'timestamp': '7:39AM'
    },
    {
        'type': 'sender_message',
        'text': "Oh, cool! F I'll have to check it out. By the way, have you tried that new restaurant that opened 
downtown?",
        'timestamp': '7:39AM'
    }
]

working on: : test_images/ss_test2.png

...


image 1/1 /Users/mac/fivv/next_sapien/yolov8/Next_sapien_V3/test_images/ss_test3.png: 832x544 27 emojis, 16 message_status_indicators, 5 receiver_messages, 1 replied_to_message, 5 sender_messages, 27 timestamps, 714.3ms
Speed: 2.6ms preprocess, 714.3ms inference, 0.4ms postprocess per image at shape (1, 3, 832, 544)


[
    {
        'type': 'receiver_message',
        'text': "That's the best feeling! Any recommendations for my next read?",
        'timestamp': '7:41AM'
    },
    {'type': 'sender_message', 'text': 'Definitely! $$  How about a gripping thriller?', 'timestamp': '7:41AM'},
    {'type': 'sender_message', 'text': 'Definitely! $$  How about a gripping thriller?', 'timestamp': '7:41AM'},
    {'type': 'receiver_message', 'text': 'Sounds intriguing! $$  Hit me with. the title.', 'timestamp': '7:42AM'},
    {
        'type': 'sender_message',
        'text': '"Gone Girl" by Gillian Flynn. Trust me, you won\'t be disappointed.',
        'timestamp': '7:42AM'
    },
    {'type': 'receiver_message', 'text': "1: Thanks! $$  I'll add it to my list", 'timestamp': '7:42AM'},
    {'type': 'receiver_message', 'text': "1: Thanks! $$  I'll add it to my list", 'timestamp': '7:42AM'},
    {
        'type': 'sender_message',
        'text': 'No problem! $$  Let me know what you think once you finish it',
        'timestamp': '7:42AM'
    },
    {'type': 'receiver_message', 'text': 'Will do! $$  Thanks for the. recommendation.', 'timestamp': '7:43AM'},
    {'type': 'sender_message', 'text': 'Anytime! $$  Happy reading!', 'timestamp': '7:43AM'}
]

working on: : test_images/ss_test4.png


image 1/1 /Users/mac/fivv/next_sapien/yolov8/Next_sapien_V3/test_images/ss_test4.png: 832x448 37 emojis, 15 message_status_indicators, 5 receiver_messages, 5 sender_messages, 29 timestamps, 589.6ms
Speed: 2.4ms preprocess, 589.6ms inference, 1.9ms postprocess per image at shape (1, 3, 832, 448)


[
    {'type': 'receiver_message', 'text': "Hey! $$  How's it going?", 'timestamp': '7:44AM'},
    {
        'type': 'sender_message',
        'text': "Hey there!. $$ It's going great,. just enjoying the sunshine. $$",
        'timestamp': '7:44 AM'
    },
    {
        'type': 'receiver_message',
        'text': "That's awesome! + I wish I could join you outside, but stuck in the office. $$",
        'timestamp': '7:44AM'
    },
    {
        'type': 'sender_message',
        'text': "Aw, bummer! we'll plan something fun for later. $$",
        'timestamp': '7:44 AM'
    },
    {
        'type': 'receiver_message',
        'text': 'Definitely!  I could use a break from all this work.',
        'timestamp': '7:44AM'
    },
    {
        'type': 'sender_message',
        'text': 'Speaking of breaks, have you tried that new coffee shop $$ . down the street?.',
        'timestamp': '7:44AM'
    },
    {
        'type': 'receiver_message',
        'text': "Not yet, but I've heard it's amazing! $$  Let's go check it out together sometime.",
        'timestamp': '7:44AM'
    },
    {
        'type': 'sender_message',
        'text': 'Sounds like a plan! $$  How about this weekend?.',
        'timestamp': '7:44 AM'
    },
    {'type': 'receiver_message', 'text': "Perfect! $$  I'll mark it on my calendar.17", 'timestamp': '7:44 AM'},
    {
        'type': 'sender_message',
        'text': "Awesome! $$  Can't wait to catch. up over some delicious coffee. $$",
        'timestamp': '7:45AM'
    }
]

# EMOJI TO UNICODE

In [ ]:
# Import Libraries

import cv2 
import pickle
import matplotlib.pyplot as plt
from pylab import array, plot, show, axis, arange, figure, uint8 
import os
from pathlib import Path
import shutil
from PIL import Image
# from skimage.measure import compare_ssim
import pickle
import glob
import matplotlib.pyplot as plt
from warnings import filterwarnings


filterwarnings('ignore')

In [ ]:
def perform_emoji_dect(emojii):
    # Resize images to a similar dimension
    # This helps improve accuracy and decreases unnecessarily high number of keypoints

    def imageResizeTrain(image):
        maxD = 1000 # 1024
        height,width = image.shape
        aspectRatio = width/height
        if aspectRatio < 1:
            newSize = (int(maxD*aspectRatio),maxD)
        else:
            newSize = (maxD,int(maxD/aspectRatio))
        image = cv2.resize(image,newSize)
        return image
    
    # Using opencv's sift implementation here

    sift = cv2.SIFT_create()

    def computeSIFT(image):
        return sift.detectAndCompute(image, None)
    
    # load emoji database
    imageList = glob.glob("emotion_data1/*.png")
    
    # We use grayscale images for generating keypoints
    imagesBW = []
    for imageName in imageList[:500]:
        # imagePath = "data/images/" + str(imageName)
        imagesBW.append(imageResizeTrain(cv2.imread(imageName,0))) # flag 0 means grayscale
        
    def fetchKeypointsandDescriptorsTest(imgpath):
        imagebw = imageResizeTrain(cv2.imread(imgpath,0))
        keypointTest, descriptorTest = computeSIFT(imagebw)
        return keypointTest, descriptorTest
    
    def fetchDescriptorFromFile(emoji):
        filepathL = os.path.abspath("DES_ALL/emotion_data1/" + emoji + "/L.txt")
        # filepathL = os.path.abspath("emotin_7k/DES_ALL/emoji/" + emoji + "/L.txt")
        descriptorL = None
        fileobjL = Path(filepathL)
        if fileobjL.exists():
            fileL = open(filepathL,'rb')
            descriptorL = pickle.load(fileL)
            fileL.close()
        # print(descriptorL)
        return descriptorL
    
    def fetchKeypointFromFile(emoji):
        filepathL = os.path.abspath("KEP_ALL/emotion_data1/" + emoji + "/L.txt")
        keypointL = []
        deserializedKeypointsL = []
        fileobjL = Path(filepathL)
        if fileobjL.exists():
            fileL = open(filepathL,'rb')
            deserializedKeypointsL = pickle.load(fileL)
            fileL.close()
        for point in deserializedKeypointsL:
            temp = cv2.KeyPoint(x=point[0][0],y=point[0][1], size=point[1], angle=point[2], response=point[3], octave=point[4], class_id=point[5])
            keypointL.append(temp)
        return keypointL
    
    def calculateScore(matches,keypoint1,keypoint2):
        return 100 * (matches/min(keypoint1,keypoint2))
    
    
    #Main KNN algorithm to detect similarity on SIFT Features
    def calculateMatches(des1,des2):
        bf = cv2.BFMatcher()

        matches = bf.knnMatch(des1,des2,k=2)
        topResults1 = []
        for i, pair in enumerate(matches):
            try:
                m, n = pair
                if m.distance < 0.705*n.distance: # 0.695
                    topResults1.append([m])
            except ValueError:
                pass  

        # print(topResults1)    
        matches = bf.knnMatch(des2,des1,k=2)
        topResults2 = []
        for i, pair in enumerate(matches):
            try:
                m, n = pair
                if m.distance < 0.740*n.distance: # 0.735
                    topResults2.append([m])
            except ValueError:
                pass

        topResults = []

        for match1 in topResults1:
            match1QueryIndex = match1[0].queryIdx
            match1TrainIndex = match1[0].trainIdx

            for match2 in topResults2:
                match2QueryIndex = match2[0].queryIdx
                match2TrainIndex = match2[0].trainIdx

                if (match1QueryIndex == match2TrainIndex) and (match1TrainIndex == match2QueryIndex):
                    topResults.append(match1)
        return topResults
    
    def findEmojiMatch(emoji):
        maxScore = float('-inf')
        matchedEmoji = None
        keypointEmoji,descriptorEmoji = fetchKeypointsandDescriptorsTest(emoji)
        #,_ = fetchDescriptorFromFile(emoji)
        # subdirs = [x[0] for x in os.walk(os.path.abspath("TEST_ALL/emotion_data1"))]
        subdirs = [x[0] for x in os.walk(os.path.abspath("TEST_ALL/emotion_data1"))]

        for subdir in subdirs[1:]:
            #r[subdir] =  {} 
            #print(subdir)
            files = os.walk(subdir).__next__()[2]    
            if (len(files) > 0):
                #print(subdir)   
                em = subdir.split('/')[-1]
                keypointL = fetchKeypointFromFile(em)
                descriptorL = fetchDescriptorFromFile(em)
                scoreL = float('-inf')
                if(keypointL and descriptorEmoji is not None and len(descriptorEmoji) > 0):
                    matchesL = calculateMatches(descriptorEmoji, descriptorL)
                    scoreL = calculateScore(len(matchesL), len(keypointEmoji), len(keypointL))

            score = scoreL
            if(score > maxScore):
                maxScore = score
                matchedEmoji = em
        # print(score,em,"**********")
        return matchedEmoji, maxScore
    
    def returnEmojiNameToUnicodeMapping():
        emoji_dict = {'angry-face': '😠', 'angry-face-with-horns': '👿', 'anguished-face': '😧', 'anxious-face-with-sweat': '😰', 
                      'astonished-face': '😲', 'beaming-face-with-smiling-eyes': '😁', 'cold-face': '🥶', 'confounded-face': '😖', 
                      'confused-face': '😕', 'cowboy-hat-face': '🤠', 'crying-face': '😢', 'Crying Face': '😭', 'disappointed-face': '😞',
                      'dizzy-face': '😵', 'downcast-face-with-sweat': '😓', 'drooling-face': '🤤', 'expressionless-face': '😑', 
                      'face-blowing-a-kiss': '😘', 'face-savoring-food': '😋', 'face-screaming-in-fear': '😱', 'face-vomiting': '🤮',
                      'face-with-hand-over-mouth': '🤭', 'face-with-head-bandage': '🤕', 'face-with-medical-mask': '😷',
                      'face-with-monocle': '🧐', 'face-with-open-mouth': '😮', 'Face with Open Mouth': '😮', 'Face with Pleading Eyes': '🥺',
                      'face-without-mouth': '😶', 'face-with-raised-eyebrow': '🤨', 'face-with-rolling-eyes': '🙄', 'face-with-steam-from-nose': '😤',
                      'face-with-symbols-on-mouth': '🤬', 'face-with-tears-of-joy': '😂', 'Face With Tears of Joy': '😂', 'face-with-thermometer': '🤒', 
                      'face-with-tongue': '😛', 'fearful-face': '😨', 'flushed-face': '😳', 'frowning-face': '😦', 'frowning-face-with-open-mouth': '😦',
                      'grimacing-face': '😬', 'grinning-face': '😀', 'grinning-face-with-big-eyes': '😃', 'grinning-face-with-smiling-eyes': '😄',
                      'grinning-face-with-sweat': '😅', 'grinning-squinting-face': '😆', 'hot-face': '🥵', 'hugging-face': '🤗', 'hushed-face': '😯', 
                      'kissing-face': '😗', 'kissing-face-with-closed-eyes': '😚', 'kissing-face-with-smiling-eyes': '😙', 'loudly-crying-face': '😭',
                      'Loudly Crying Face': '😭', 'lying-face': '🤥', 'money-mouth-face': '🤑', 'nauseated-face': '🤢', 'nerd-face': '🤓', 'neutral-face': '😐'
                      , 'partying-face': '🥳', 'Partying Face': '🥳', 'pensive-face': '😔', 'persevering-face': '😣', 'pleading-face': '🥺', 'pouting-face': '😡', 
                      'relieved-face': '😌', 'sad-but-relieved-face': '😥', 'shushing-face': '🤫', 'sleeping-face': '😴', 'sleepy-face': '😪', 'slightly-frowning-face': '🙁', 
                      'slightly-smiling-face': '🙂', 'smiling-face': '😊', 'smiling-face-with-halo': '😇', 'smiling-face-with-heart-eyes': '😍', 'smiling-face-with-hearts': '🥰', 
                      'smiling-face-with-horns': '😈', 'smiling-face-with-smiling-eyes': '😊', 'Smiling Face with Smiling Eyes': '😊', 'smiling-face-with-sunglasses': '😎',
                      'smirking-face': '😏', 'sneezing-face': '🤧', 'squinting-face-with-tongue': '😝', 'thinking-face': '🤔', 'tired-face': '😫', 'unamused-face': '😒',
                      'upside-down-face': '🙃', 'weary-face': '😩', 'winking-face': '😉', 'winking-face-with-tongue': '😜', 'woozy-face': '🥴', 'worried-face': '😟', 
                      'yawning-face': '\U0001f971', 'zany-face': '🤪', 'zipper-mouth-face': '🤐', 'phone-receiver': '📞', 'sad-face': '😢', 'loudly-laughing-face':'😂',
                      'Weary Face':'😩', 'face with hand over mouth':'🤭', 'Face Blowing A Kiss':'😘', 'Face With Hand Over Mouth Emoji':'🤭', 'Partying-Face': '🥳',
                     'new-moon-face':'🌚', 'saluting-face':'🫡', ' Face with Peeking Eye':'🫣', 'Grinning Face with Sweat':'😅', 'Face Throwing a Kiss':'😘',
                     'Face-Throwing-a-Kiss':'😘', "Grinning Face":'😀' , "Grinning Face":'😀', "Confused Face": '😕', 'Smiling Face with Tear': '🥲',
                     'Smiling-Face-with-Tear':"🥲", "Confused-Face": '😕', 'Winking Face with Tongue':'😜', 'Winking-Face-with-Tongue':'😜',
                     'fire':'🔥', 'folded-hand': '🙏'}
        return emoji_dict
    
    
    
    emojiMappingInImg = {}

    # all_emoji = glob.glob("/home/tim/jupyterNotes/fivv/next_sapien/yolov8/cropped/emoji/*jpg")
    # print(all_emoji)

    # def pred_emo(emojii):

    for emoji in emojii:
        matchedEmoji, maxScore = findEmojiMatch(emoji)
        print(maxScore)
        if(matchedEmoji != None and maxScore >= 8):
            try:
                unicodeEmoji = returnEmojiNameToUnicodeMapping()[matchedEmoji]
                realEmoji = emoji.split('/')[-1]  # Extracting real emoji name from file path
                print(f"Real Emoji: {realEmoji}, Predicted Emoji: {unicodeEmoji}, Match Score: {maxScore}")
                real_emo = plt.imread(os.path.join("cropped", "emoji", realEmoji))
                plt.imshow(real_emo)
                plt.title(f"Pred Emoji: {unicodeEmoji}")
                plt.axis('off')
                save_pth = "Results"
                os.makedirs(save_pth, exist_ok=True)
                plt.savefig(f"{save_pth}/result_{realEmoji}")
                plt.show()
            except KeyError as K:
                pass
    return
    

In [ ]:
all_emo = glob.glob("cropped/emoji/*jpg")
perform_emoji_dect(all_emo)